<a href="https://colab.research.google.com/github/dav3794/CeNT_internship_Dawid_Uchal/blob/master/topoly_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install topoly
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 11.0 MB/s 
     |████████████████████████████████| 2.3 MB 49.2 MB/s 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=86840d8f6d4fc275565c10afcfec9222ed6141a8f0382a13df496ffd74c4d46e
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import json
from tqdm import tqdm

def fasta_to_json(file, save_json=0):
    header = ['id', 'sequence']
    data = {}
    seq_temp = ""

    with open(f"{file}_full_length_sequences.fasta", "r") as input_file:
        for line in input_file.readlines():
            if line[0] == ">":
                if len(seq_temp) > 0:
                    data.update({entry: seq_temp})
                seq_temp = ""
                pattern = '>.*? '
                match_results = re.search(pattern, line, re.IGNORECASE)
                entry = match_results.group()
                entry = re.sub(">", "", entry)
                entry = entry.strip()
            else:
                seq_temp += line.strip()
        else:
            data.update({entry: seq_temp})
            #print("Done")

    if save_json:
        json_dict = json.dumps(data)
        with open(f"{file}_sequences.json", "w") as f:
            f.write(json_dict)
            print(f"Saved to {file}_sequences.json")
    return data


def atomize_clusters(file, save=True):
    cwd = os.getcwd()
    dir = os.path.join(cwd, f"{file}_clusters")
    if not os.path.exists(dir):
        os.mkdir(dir)

    seq_dict = fasta_to_json(f"PF{file}")
    out = []
    i = 0
    with open(f"{file}.clstr", "r") as f:
        for line in tqdm(f.readlines()):
            if line[0] == ">":
                if i > 0:
                    if save:
                        pd.DataFrame(clstr).drop_duplicates().to_csv(f"{dir}/Cluster {i-1}.txt", sep=",", header=["id", "sequence"], index=False)
                    else:
                        out.append(clstr)
                i += 1
                clstr = []
            else:
                items = line.split()
                entry = items[2][1:-3]
                row = [[entry, seq_dict[entry]]]
                if items[-1] == "*":
                    clstr = row + clstr
                else: clstr += row
        else:
            if save:
                pd.DataFrame(clstr).drop_duplicates().to_csv(f"{dir}/Cluster {i-1}.txt", sep=",", header=["id", "sequence"], index=False)
            else:
                out.append(clstr)
                return out


In [ ]:
import multiprocessing
import multiprocessing.pool

class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [ ]:
import wget
import shutil
from topoly import alexander, homfly
from glob import glob


def get_pLDDT(path):
    with open(path, "r") as f:
              cif = f.read()
              pattern = 'global\.metric_value .*?\n'
              match_results = re.search(pattern, cif, re.IGNORECASE)
              entry = match_results.group()
              return float(entry.split()[1])


def calculate_topology(file, print_knot = True, from_saved = True, use_matrix=False):
    if from_saved:
        df = pd.read_csv(file, sep=",", header=0)
        df = df.to_numpy()
    else:
        df = file

    if not os.path.exists("./data/"):
      os.mkdir("./data/")

    for id, sequence in df:
        try:
          id_AF = re.sub("_.*?$", "", id)
          path = f"./data/{id_AF}.cif"
          url = f"https://alphafold.ebi.ac.uk/files/AF-{id_AF}-F1-model_v3.cif"
          if not os.path.exists(path):
            wget.download(url, out=path)

          if get_pLDDT(path) > 0.5:
              if use_matrix:
                  knot = alexander(path, tries=50, matrix=True, density=3)
              else:
                  knot = homfly(path, max_cross=50, translate=True)

              knot_type = "01"
              if "0_1" in knot:
                  if knot["0_1"] > 0.5:
                      if print_knot:
                          print(id, "unknot")
                  else:
                      tops = list(knot.keys())
                      tops.sort(key=lambda i: knot[i], reverse=True)
                      knot_type = re.sub('[^A-Za-z0-9]+', '', tops[0])
                      if print_knot:
                          print(id, "knot: " + knot_type)
                  return [id, sequence, knot_type]
        except: 
            if print_knot:
                print("Error occured", id)
            else: pass


def get_topology(family_id, multiprocess = 1, seed = 0, use_matrix=False):
    if multiprocess:
        print("Atomizing clusters to files:")
        atomize_clusters(family_id)
        print("Done.")

        files = os.path.join(f"./{family_id}_clusters/", "Cluster *.txt")
        files = glob(files)

        if seed:  files = files[:seed]

        print("Calculating topology for protein:")
        with MyPool(10) as p:
            data = p.map(calculate_topology, files)

        data = list(filter(None, data))
        
        print("Deleting redundant data.")
        dir_path = f'./{family_id}_clusters/'
        try:
            shutil.rmtree(dir_path)
            #shutil.rmtree('./data/')
            print("Data removed.")
        except OSError as e:
            print("Error: %s : %s" % (dir_path, e.strerror))

    else:
        print("Atomizing clusters.")
        out = atomize_clusters(family_id, 0, 0)
        if seed:   out = out[:seed]
        data = []

        print("Calculating topology for proteins:")
        for file in tqdm(out):
            res_tmp = calculate_topology(file, 0, use_matrix = use_matrix)
            if res_tmp: data.append(res_tmp)

    print("Saving calculated topology.")
    pd.DataFrame(data).to_csv(f"{family_id}_topology.csv", sep=",", header=["id", "seq", "type_of_knot"], index=False)
    print(f"Topology saved as {family_id}_topology.csv")

    return data

Do prawidłowego uruchomienia należy w miejscu wykonywania programu umieścić plik "PF`{family_id}`_full_length_sequences.fasta" oraz plik wyjściowy .clstr z CD-HIT nazwany "`{family_id}`.clstr".

In [ ]:
if __name__ == '__main__':
    data = get_topology("04013")

In [ ]:
!zip -r dane.zip ./data